# Préparation des intrants et simulations à large échelle dans HEC-ResSim

### Réalisé par Sébastien Langlois

In [1]:
dss_directory = '/media/slanglois/ssd-2tb/laminage/dss/*.dss'
csv_directory = '/media/slanglois/ssd-2tb/laminage/csv/*.csv'
output_path_dss = '/media/slanglois/ssd-2tb/laminage/dss/'
output_path_alt = '/media/slanglois/ssd-2tb/laminage/alternatives'

type_series = 'STO'

nom_BV_hec=['Inflow Mitchinamecus','Inflow Kiamika','Inflow Mont-Laurier',
    'Inflow Cedar','Inflow High Falls','Inflow Petite Nation','Inflow Masson',
    'Inflow Rideau','Inflow South Nation','Inflow Cabonga','Inflow Baskatong',
    'Inflow Paugan','Inflow Arnprior','Inflow Mountain Chute','Inflow Kamaniskeg',
    'Inflow Bark Lake','Inflow Dumoine','Inflow Mattawa',
    'Inflow Joachims','Inflow Otto Holden','Inflow Kipawa','Inflow Dozois',
    'Inflow Victoria','Inflow Rapide 7','Inflow Rapide 2','Inflow Rabbit Lake',
    'Inflow Lower Notch','Inflow Lady Evelyn','Inflow Mistinikon','Inflow Blanche',
    'Inflow Temiscamingue','Inflow Maniwaki','Inflow Chelsea','Inflow Carillon',
    'Inflow Kinojevis','Inflow des Quinze (Anglier)','Inflow Petawawa','Inflow Chenaux et Noire',
    'Inflow Coulonge','Inflow Bonnechere','Inflow Chat Falls','Inflow Mississippi',
    'Inflow Rouge']

nom_BV_dss = ['MITCHINAMECUS','KIAMIKA','MONT-LAURIER','LAC DU POISSON BLANC',
    'HIGH FALLS','RIVIERE PETITE NATION','MASSON','RIVIERE RIDEAU',
    'RIVIERE SOUTH NATION','CABONGA','BASKATONG','PAUGAN',
    'MADAWASKA-ARNPRIOR','MOUNTAIN CHUTE','KAMANISKEG','BARK LAKE',
    'RIVIERE DUMOINE','RIVIERE MATTAWA','DES JOACHIMS','OTTO HOLDEN',
    'KIPAWA','DOZOIS','LAC VICTORIA ET LAC GRANET','RAPIDE 7',
    'RAPIDE 2','RABBIT LAKE','LOWER NOTCH ET INDIAN CHUTE','LADY EVELYN',
    'MISTINIKON','RIVIERE BLANCHE','LAC TEMISCAMINGUE A ANGLIERS','MANIWAKI',
    'CHELSEA','CARILLON ET HULL','RIVIERE KINOJEVIS','LAC DES QUINZE',
    'RIVIERE PETAWAWA','CHENAUX ET NOIRE','RIVIERE COULONGE','RIVIERE BONNECHERE',
    'CHUTE-DES-CHATS','RIVIERE MISSISSIPPI','RIVIERE ROUGE']

keys_link = dict(zip(nom_BV_hec, nom_BV_dss))

reseau_euler = False

In [2]:
import pandas as pd
import glob
import sys
import os
from dask.distributed import Client, progress
import dask
from dask_jobqueue import SLURMCluster
from dask.diagnostics import ProgressBar

# Local functions
sys.path.append(os.path.abspath(os.path.join('..')))
import laminage as lm

In [3]:
if reseau_euler:
    # à modifier lorsque dans le réseau Euler/Euler2
    cluster = SLURMCluster(cores=8,
                           processes=4,
                           memory="16GB",
                           project="woodshole",
                           walltime="01:00:00",
                           queue="normal")
    client = Client(cluster)
else:
    client = Client()

client

Client Scheduler: tcp://127.0.0.1:33839 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 8 Cores: 32 Memory: 135.11 GB


## Conversion des fichiers csv vers dss

In [ ]:
lazy_results = [dask.delayed(lm.csv_to_dss)(filename, output_path) 
                for filename in glob.glob(csv_directory)]
dask.compute(*lazy_results)

## Conversion des fichiers dss vers les alternatives

In [4]:
alternatives_obj_list = [lm.CreationAlternative(dss_filename, output_path, type_series, keys_link) 
                         for dss_filename in glob.glob(dss_directory)]

lazy_results = [dask.delayed(alternative.create_all())
                for alternative in alternatives_obj_list]
dask.compute(*lazy_results)